In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pickle

In [7]:
import tensorflow_datasets as tfds
dataset, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)
train_data, test_data = dataset['train'], dataset['test']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\ALIHASSAN\tensorflow_datasets\imdb_reviews\plain_text\incomplete.ASQTMJ_1.0.0\imdb_reviews-…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\ALIHASSAN\tensorflow_datasets\imdb_reviews\plain_text\incomplete.ASQTMJ_1.0.0\imdb_reviews-…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\ALIHASSAN\tensorflow_datasets\imdb_reviews\plain_text\incomplete.ASQTMJ_1.0.0\imdb_reviews-…

Dataset imdb_reviews downloaded and prepared to C:\Users\ALIHASSAN\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [ ]:
train_texts = []   #3list of reviews
train_labels = []    ##Label of reviews(0/1)
for text, label in train_data:
    train_texts.append(text.numpy().decode('utf-8'))
    train_labels.append(label.numpy())

In [9]:
test_texts = []
test_labels = []
for text, label in test_data:
    test_texts.append(text.numpy().decode('utf-8'))
    test_labels.append(label.numpy())

In [10]:
# Parameters
vocab_size = 100000
maxlen = 100

In [11]:
# Tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

In [12]:
# Save tokenizer
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)


In [13]:
x_train = tokenizer.texts_to_sequences(train_texts)
x_test = tokenizer.texts_to_sequences(test_texts)


In [14]:
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
y_train = np.array(train_labels)
y_test = np.array(test_labels)

In [19]:
model = Sequential([
    Embedding(vocab_size, 64, input_length=maxlen),
    Bidirectional(GRU(128, dropout=0.3, recurrent_dropout=0.3)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])


In [20]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

In [22]:
model.fit(x_train, y_train, epochs=5, batch_size=128, validation_data=(x_test, y_test), callbacks=[early_stop])

Epoch 1/5
196/196 [==============================] - 183s 892ms/step - loss: 0.5306 - accuracy: 0.7092 - val_loss: 0.3750 - val_accuracy: 0.8403
Epoch 2/5
196/196 [==============================] - 148s 756ms/step - loss: 0.2564 - accuracy: 0.8994 - val_loss: 0.3673 - val_accuracy: 0.8423
Epoch 3/5
196/196 [==============================] - 153s 780ms/step - loss: 0.1435 - accuracy: 0.9489 - val_loss: 0.4239 - val_accuracy: 0.8358
Epoch 4/5
196/196 [==============================] - 162s 825ms/step - loss: 0.0845 - accuracy: 0.9714 - val_loss: 0.6266 - val_accuracy: 0.8258


In [23]:
loss, acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {acc:.2f}")

782/782 [==============================] - 34s 43ms/step - loss: 0.3673 - accuracy: 0.8423
Test Accuracy: 0.84


In [25]:
model.save("sentiment_model.h5")